In [40]:
import urbandictionary as ud
import re
import json
import pytrends
from pytrends.request import TrendReq
import wikipedia
import requests
from bs4 import BeautifulSoup
import random
import PyDictionary
from PyDictionary import PyDictionary
#import spacy
import pandas as pd
import re
import time

bad_patterns = r"(rapist*)|(christ)|(ass)|(sex*)|(hoe)|(cock*)|(mastu)|(damn*)|(slut*)|(church)|(christians)|(pedo*)|(titt*)|(jesus)|(asse*)|(israel*)| (fellatio)|(genit*)|(christianity)|(atheists)| (masterb*)|(gay)|(bible)|(muslim)| (bitch*)|(jew*)|(nipple*)|(religion)| (dick*)| (ho )|(praye)|(islam)|(jugs)|(atheism)|(tit*)|(rape*)|(atheist)| (bastard*)|(penis)|(yeshua)| (puss*)|(whor*)|(fagg*)|(allah)|(erotic*)|(balls)|(penal)|(fuck*)|(christian)| (fuck)| (nigg*)|(boob*)|(peni*)|(boob)|(ejaculate)|(lord)|(shit)|(faith)| (cunt*)|(islamic)| (breast*)|(yahwey)|(nipples)|(blowjob)| (nud*)|(breast*)| (porn*)| (naked)|(pray)| (nipple*)|(-ass*)|(lynch)|(nigg*)|(god)"

# Split the string into a list of words
bad_patterns_list = re.split(r'\|', bad_patterns)

# Remove duplicates from the list
bad_patterns_list = list(set(bad_patterns_list))

# Join the list back into a string
bad_patterns = '|'.join(bad_patterns_list)

print(bad_patterns)

replacements = {
    '\bsex*': 'affection',
    '\bporn*': 'adult entertainment',
    '\bfuck*': 'intercourse',
    '-ass*': 'rear end',
    'ass': 'rear end',
    '\bshit': 'feces',
    'damn*': 'darn',
    'ass|asse*': 'rear end',
    'cock*': 'male bird',
    'whor*': 'prostitute',
    'nigg*': 'racial slur',
    'slut*': 'promiscuous person',
    'blowjob': 'oral sex',
    'fagg*': 'homosexual',
    'boob|boob*': 'breast',
    'bitch*': 'mean woman',
    'bastard*': 'illegitimate child',
    'ho |hoe': 'prostitute',
    'breast*|jugs': 'chest',
    'cunt*': 'vulva',
    'puss*': 'vagina',
    'dick*': 'penis',
    'naked': 'unclothed',
    'nud*': 'unclothed',
    'masterb*': 'self-gratification',
    'mastu': 'self-gratification',
    'god':'deity',
    'jesus':'religious figure',
    'christ':'religious figure',
    'bible':'religious text',
    'church':'place of worship',
    'religion':'set of beliefs',
    'pray':'to communicate with a deity',
    'prayer':'a conversation with a deity',
    'faith':'belief',
    'lord':'captain',
    'allah':'a diety',
    'gay':'homosexual',
    'rapist*|rape*': 'sexual assault',
    'pedo*': 'child sexual abuse(r)',
    'yahwey':'a deity',
    'yeshua':'a religious figure'
}

def replacer_censor(definition,phrase,replacements_dict):
    # Iterate over the keys in the replacements dictionary
    for pattern in replacements_dict:
        # Use re.sub to replace the occurrences of the pattern with its corresponding value in the phrase and definition strings
        phrase = re.sub(pattern, replacements_dict[pattern], phrase)
        definition = re.sub(pattern, replacements_dict[pattern], definition)
    return phrase, definition

def check_for_badwords(definition, phrase, bad_patterns, replacements_dict):
    # Replace the bad words with their corresponding replacements
    phrase, definition = replacer_censor(definition, phrase, replacements_dict)

    # Check if the phrase or definition contain any of the bad patterns
    if any(re.search(r'\b' + word + r'\b', definition) for word in bad_patterns) or any(re.search(r'\b' + word + r'\b', phrase) for word in bad_patterns):
        return True
    else:
        return False


def check_for_good_patterns(definition, title):
    # check both title and definition for good patterns
    good_patterns = [r'\b\w+phobia\.?\b', r'\bslang\b', r'\bacronymn\b', r'\bmeme\b']


    if any(re.match(r'\b' + word + r'\b', definition) for word in good_patterns):
        print(f'Found a good pattern in the definition: {definition}')
        return True
    elif any(re.match(r'\b' + word + r'\b', title) for word in good_patterns):
        print(f'Found a good pattern in the title: {title}')
        return True
    else:
        return False

def remove_undesireable_sentences(definition, title, bad_patterns):
    # convert the definition and title to lowercase
    definition = definition.lower()
    title = title.lower()

    # split the definition into sentences
    sentences = definition.split('.')

    # split the title into words
    title_words = title.split()

    # initialize a list to store the acceptable sentences
    acceptable_sentences = []

    # iterate over the sentences in the definition
    for sentence in sentences:
        # split the sentence into words
        sentence_words = sentence.split()

        # check if any of the words in the sentence are also in the title
        if not any(word in title_words for word in sentence_words):
            # if not, add the sentence to the list of acceptable sentences
            acceptable_sentences.append(sentence)

    # remove sentences that are not in English
    definition = re.sub(r'[^\x00-\x7f]',r'', '.'.join(acceptable_sentences))
    # remove sentences that contain a regex match to any word in the buzzwords list
    definition = re.sub(r'|'.join(map(re.escape, bad_patterns)), '', definition)
    return definition

def unpack_definitions(definition):
    # remove the brackets and clean up the definitions
    # with regex
    definition = definition.replace("[","")
    definition = definition.replace("]","")
    definition = definition.replace("'","")
    definition = definition.replace('"',"")
    definition = definition.replace("(","")
    definition = definition.replace(")","")

    # remove double spaces
    definition = definition.replace('  ',' ')

    return definition

def get_page_length(phrase):
  # search for pages on Wikipedia that match the given phrase
    try:
        pages = wikipedia.search(phrase)

        # retrieve the first page from the search results
        page = wikipedia.page(pages[0])
        print(f'Found the page for {page.title}', end='')
        if phrase.lower() != page.title.lower():
            print(' ... nevermind... not the right page.')
            return 0
        # return the length of the page
        return len(page.content)
    except Exception as e:
            return 0


def main():
    # Set up variables
    wikitest = False # set to true to test the wikipedia page length
    rand_dict = {}
    total_votes_thresh = 100 # min number of upvotes + downvotes
    upvotes_thresh = 100 # min number of upvotes for a single word to count  as a card
    downvotes_thresh = upvotes_thresh//2 # max number of downvotes
    desired_number_of_cards = 30 # number of cards to generate
    rands = [] # the randoms

    # Get random phrases and definitions from Urban Dictionary until desired number of cards is reached
    while len(rand_dict) < desired_number_of_cards:
        time.sleep(1)
        rand = ud.random() # returns a list of 5 random phrases and definitions from Urban Dictionary
        rands.extend(rand) # rands is a list of all the random phrases and definitions from Urban Dictionary
        for element in rand:
            phrase = element.word
            definition = element.definition
            upvotes = element.upvotes
            downvotes = element.downvotes
            values = [upvotes + downvotes >= total_votes_thresh,
                      upvotes >= upvotes_thresh and downvotes <= downvotes_thresh,
                      check_for_good_patterns(definition, phrase),
                      len(phrase.split()) > 1] # check if the phrase is more than one word (final check)

            # Check if the element meets the criteria to be added to the dictionary
            if any(values) and not check_for_badwords(definition, phrase, bad_patterns, replacements):
                # Clean up and format the definition
                definition = unpack_definitions(definition)
                definition = remove_undesireable_sentences(definition, phrase, bad_patterns)
                # Add the element to the dictionary
                print(f'{upvotes} | Added {phrase}: {definition[0:10]}...')
                rand_dict[phrase] = definition
    # Save the dictionary to a CSV and JSON file
    df = pd.DataFrame.from_dict(rand_dict, orient='index')
    df.to_csv('cards.csv', header=False)
    with open('cards.json', 'w') as f:
        json.dump(rand_dict, f)


    # Hippopotomonstrosesquippedaliophobia - fear of long words
    # "nachlophobia": "the fear that your ultimate connection has people are actually pretty shallow and even though you consider them close you refuse to let them see the real you use of fear of you losing them as a repercussion of your fears, hopes and dreams.

(rapist*)|(christ)|(ass)|(sex*)|(hoe)|(cock*)|(damn*)|(slut*)|(titt*)|(christians)|(pedo*)| (fellatio)|(jesus)|(asse*)|(genit*)|(christianity)|(atheists)| (masterb*)|(gay)|(bible)|(muslim)| (bitch*)|(jew*)|(nipple*)|(religion)| (dick*)| (ho )|(praye)|(islam)|(-ass*)|(jugs)|(atheism)|(tit*)|(rape*)|(atheist)| (bastard*)|(penis)|(yeshua)| (puss*)|(whor*)|(fagg*)|(allah)|(erotic*)|(balls)|(god)|(penal)|(fuck*)|(christian)| (fuck)| (nigg*)|(boob*)|(peni*)|(boob)|(ejaculate)|(lord)|(shit)|(nigg*)|(faith)| (cunt*)|(islamic)| (breast*)|(yahwey)|(nipples)|(blowjob)| (nud*)|(breast*)| (porn*)| (naked)|(pray)| (nipple*)|(mastu)|(lynch)|(church)|(israel*)


In [42]:
main()

error: missing ), unterminated subpattern at position 2